# Plot Pareto Fronts

Plot the pareto fronts from an optimization's logbook, assuming that the [DEAP framework](https://github.com/DEAP/deap) was used.

This code is also part of the [SMOC project](https://github.com/mdmfernandes/smoc).

The last version of this document is available [here](https://github.com/mdmfernandes/smoc-extras).

## Author

* **Miguel Fernandes** - [mdmfernandes @ GitHub](https://github.com/mdmfernandes)

In [1]:
# Imports
%matplotlib inline
import array
import pickle
import random
import matplotlib.pyplot as plt
import math

from deap import algorithms, base, creator, tools

from bokeh.models import PrintfTickFormatter
from bokeh.palettes import viridis
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.io import output_notebook

In [2]:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))
# Define an individual
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

# random generated float
toolbox.register("attr_float", random.uniform, 0, 1)
# Define an individual as a list of floats (iterate over "att_float"
# and place the result in "creator.Individual")
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
# Define the population as a list of individuals (the # of individuals
# is only defined when the population is initialized)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


fname = "../files/lb_20190212_10-55.pickle"

with open(fname, 'rb') as f:
        logbook = pickle.load(f)
        
# Population from all generations
population = logbook.chapters['population'].select('value')
# Population from the last generation
result = population[-1]

# Pareto fronts from the last generation's population
fronts = tools.emo.sortLogNondominated(result, len(result))

In [3]:
circuit_vars = {
    'inv_vth': [[0.3, 0.8], 'V'],
    'filt_cap': [[100e-15, 50e-12], 'F'],
    'filt_len': [[0.2, 0.56], 'um'],
    'filt_len_b': [[0.2, 0.56], 'um'],
    'filt_len_casn': [[1, 40], 'um'],
    'filt_len_casp': [[1, 40], 'um'],
    'filt_res_casn': [[5e3, 30e3], 'R'],
    'filt_res_casp': [[5e3, 30e3], 'R'],
    #'filt_res_vbp': [[1e3, 10e3], 'R'],
    'filt_wn_bl': [[1, 35], 'um'],
    'filt_wn_br': [[1, 100], 'um'],
    'filt_wn_casn': [[1, 50], 'um'],
    'filt_wn_tl': [[1, 50], 'um'],
    'filt_wn_tr': [[1, 150], 'um'],
    'filt_wp_bl': [[1, 5], 'um'],
    'filt_wp_br': [[1, 150], 'um'],
    'filt_wp_casp': [[1, 50], 'um'],
    'filt_wp_tl': [[1, 50], 'um'],
    'filt_wp_tr': [[1, 150], 'um'],
    'filt_wp_vbp': [[1, 35], 'um'],
    'idac': [[0.1e-6, 25e-6], 'A'],
    'idac2': [[0.1e-6, 50e-6], 'A'],
    'Cosc': [[1e-15, 250e-15], 'F'],
    'osc_len_bias': [[1, 20], 'um'],
    'osc_len_btm': [[0.2, 0.56], 'um'],
    'osc_len_top': [[0.2, 0.56], 'um'],
    'osc_wn': [[1, 20], 'um'],
    'osc_wn_bb': [[1, 50], 'um'],
    'osc_wn_bias': [[1, 20], 'um'],
    'osc_wn_bt': [[1, 50], 'um'],
    'osc_wp': [[1, 50], 'um'],
    'Rbias': [[1e3, 25e3], 'R'],
    'Rosc': [[1e3, 50e3], 'R']
}

objectives = {
    'POWER': [-1.0, 'W'],
    'SINAD': [4.0, 'dB']
}

constraints = {
    'SINAD': [[70, None], 'dB']
}

In [4]:
def eng_string(x, sig_figs=3, si=True):
    """Returns the input value formatted in a simplified engineering
    format, i.e. using an exponent that is a multiple of 3.

    Copied from: link<https://stackoverflow.com/questions/17973278/python-decimal
                 -engineering-notation-for-mili-10e-3-and-micro-10e-6>

    Arguments:
        x (float or int): value to format.

    Keyword Arguments:
        sig_figs (int, optional): number of significant digits (default: 3).
        si (boolean, optional): use SI suffix for exponent, e.g. k instead of
            e3, n instead of e-9 etc. (default: True).

    Returns:
        str: the formatted value.
    """

    x = float(x)
    sign = ''
    if x < 0:
        x = -x
        sign = '-'
    if x == 0:
        exp = 0
        exp3 = 0
        x_3 = 0
    else:
        exp = int(math.floor(math.log10(x)))
        exp3 = exp - (exp % 3)
        x_3 = x / (10 ** exp3)
        x_3 = round(x_3, -int(math.floor(math.log10(x_3)) - (sig_figs-1)))
        if x_3 == int(x_3):  # prevent from displaying .0
            x_3 = int(x_3)

    if si and exp3 >= -24 and exp3 <= 24 and exp3 != 0:
        exp3_text = 'yzafpnum kMGTPEZY'[exp3 // 3 + 8]
    elif exp3 == 0:
        exp3_text = ''
    else:
        exp3_text = 'e%s' % exp3

    return ('%s%s%s') % (sign, x_3, exp3_text)


In [5]:
def plot_pareto_fronts(fronts, circuit_vars, objectives, constraints, plot_fname):
    """Plot the pareto fronts given by the optimizer.

    Arguments:
        fronts (list): pareto fronts.
        circuit_vars (dict): circuit design variables w/ units.
        objectives (dict): circuit optimization objectives w/ units.
        constraints (dict): circuit optimization constraints.
        plot_fname (str): path of the plot file.
    """
    vars_names = list(circuit_vars.keys())
    vars_units = [var[1] for var in circuit_vars.values()]

    fit_names_tmp = list(objectives.keys())
    fit_names = [f"{name}_fit" for name in fit_names_tmp]
    fit_units = [fit[1] for fit in objectives.values()]

    sim_res = {**objectives, **constraints}
    
    
    sim_res_names = list(sim_res.keys())
    sim_res_units = [res[1] for res in sim_res.values()]
           
    # Define the colors to use in the graphic, according to the number of pareto fronts
    # each front = one color
    num_colors = max(3, len(fronts))
    try:
        colors = viridis(num_colors)
    except KeyError as err:
        raise KeyError(f"The colors vector doesn't support {err} colors.")


    # Configure the tooltips
    tooltips = """
        <style>
            .bk-tooltip>div:not(:first-child) {display:none;}
            .bk-tooltip>div {color: @valid;}
        </style>
        <div>
    """

    # Add the fitnesses to the tooltips
    #tooltips += '<div style="font-weight: bold; font-size: 1.4rem;">Fitness</div>'
    #for fit_tmp, fit in zip(fit_names_tmp, fit_names):
    #    tooltips += f"<i>{fit_tmp}:</i> @{fit}<br>\n"
    tooltips += f"x: @{fit_names[0]}, y: @{fit_names[1]}"
    
    # Add all the simulation results to the tooltips
    tooltips += '<div style="font-weight: bold; font-size: 1.4rem;">Results</div>'
    for res in sim_res_names:
        tooltips += f"<i>{res}:</i> @{res}<br>\n"

    # Add the variables to the tooltips
    tooltips += '<div style="font-weight: bold; font-size: 1.4rem;">Variables</div>'
    for var in vars_names:
        tooltips += f"<i>{var}:</i> @{var}<br>\n"
        
    tooltips += "</div>"
        
    output_notebook()

    # Create the figure
    date_time = plot_fname.split('/')[-1].split('.')[0]
    title = f"Plotting {len(fronts)} pareto fronts - {date_time.replace('-', ':')}"
    p = figure(plot_width=600, plot_height=600, tooltips=tooltips, title=title,
               active_scroll='wheel_zoom')

    for idx, front in enumerate(fronts):
        # Get the fitness values from the pareto front
        fits = list(map(lambda ind: ind.fitness.values, front))
        # Get the simulation results from the pareto front
        sim_res = list(map(lambda ind: ind.result, front))

        # The '*' separates the various fitnesses, otherwise it will try to zip
        # all the fitnesses at the same time and put them in x,y, which will give
        # an error if we have more than two fitness (and even with 2 the result
        # won't be the expected)
        (x, y) = zip(*fits)  # Unpack the fitnesses

        # Scale the power to uW
        #x = [val*1e6 for val in x]
        # or x = list(map(lambda x: x*1e6, x))
        # List comprehension is more pythonic and it's usually faster
        # if we need to use lambdas in map

        # Values to plot on the graphic
        source = dict(x=x, y=y)

        # Add the fitnesses to the tooltips
        for j, fit in enumerate(fit_names_tmp):
            source.update({f"{fit}_fit": [f"{eng_string(f[j])}" for f in fits]})
        
        # Add the simulation results to the tooltips
        for j, res in enumerate(sim_res_names):
            source.update({res: [f"{eng_string(r[res])}{sim_res_units[j]}" for r in sim_res]})

        # Add the circuit variables to the tooltips
        # ind[j] é a variável 'j' do individuo
        for j, var in enumerate(vars_names):
            source.update({var: [f"{eng_string(ind[j])}{vars_units[j]}" for ind in front]})
            
        valid = ['black']*len(front)
        
        # Check if an individual is valid (if all the constraints are fulfilled)
        for ind in range(len(front)):
            for pos, val in enumerate(fit_names_tmp):
                fit = fits[ind]
                res = sim_res[ind]
                if fit[pos] != res[val]:
                    valid[ind] = 'red'
                    
        source.update({'valid': [val for val in valid]})

        # Plot the graphic
        p.circle('x', 'y', source=ColumnDataSource(data=source), size=10,
                 color=colors[idx], muted_color=colors[idx], muted_alpha=0.1,
                 legend=f"Pareto {idx+1} (ind={len(front)})")

    # Format the title
    p.title.text_font_size = '16pt'
    p.title.align = 'center'
    # Format the axis labels
    p.xaxis.axis_label = f"{fit_names[0]} [{fit_units[0]}]"
    p.xaxis.formatter = PrintfTickFormatter(format='%.2e')
    p.yaxis.axis_label = f"{fit_names[1]} [{fit_units[1]}]"
    p.axis.axis_label_text_font_style = 'bold'
    p.axis.axis_label_text_font_size = '11pt'
    # Format the legend
    p.legend.location = "bottom_right"
    p.legend.click_policy = "mute"
    p.legend.label_text_font_size = '8pt'

    show(p)

## Plot Pareto Fronts

In [6]:
plot_pareto_fronts(fronts, circuit_vars, objectives, constraints, plot_fname='test new')

Loading BokehJS ...

In [7]:
def plot_pareto_fronts_old(fronts, circuit_vars, objectives, plot_fname):
    """Plot the pareto fronts given by the optimizer.
    Arguments:
        fronts (list): pareto fronts.
        circuit_vars (dict): circuit design variables w/ units.
        objectives (dict): circuit optimization objectives w/ units.
        plot_fname (str): path of the plot file.
    """
    vars_names = list(circuit_vars.keys())
    vars_units = [var[1] for var in circuit_vars.values()]

    fit_names = list(objectives.keys())
    fit_units = [fit[1] for fit in objectives.values()]

    # Define the colors to use in the graphic, according to the number of pareto fronts
    # each front = one color
    num_colors = max(3, len(fronts))
    try:
        colors = viridis(num_colors)
    except KeyError as err:
        raise KeyError(f"The colors vector doesn't support {err} colors.")

    # Add the Fitnesses to the tooltips
    tooltips = [(fit, f"@{fit}") for fit in fit_names]

    # Add separator between fitnesses and circuit_variables
    tooltips.extend([(":::::::::::::::", "::::::::::::::")])

    # Add the variables to the tooltips
    tooltips.extend([(var, f"@{var}") for var in vars_names])

    output_notebook()
    
    # Create the figure
    date_time = plot_fname.split('/')[-1].split('.')[0]
    title = f"Plotting {len(fronts)} pareto fronts - {date_time.replace('-', ':')}"
    p = figure(plot_width=600, plot_height=600, tooltips=tooltips, title=title,
               active_scroll='wheel_zoom')

    for idx, inds in enumerate(fronts):
        # Get the fitness values
        fits = list(map(lambda ind: ind.fitness.values, inds))

        # The '*' separates the various fitnesses, otherwise it will try to zip
        # all the fitnesses at the same time and put them in x,y, which will give
        # an error if we have more than two fitness (and even with 2 the result
        # won't be the expected)
        (x, y) = zip(*fits)  # Unpack the fitnesses

        # Scale the power to uW
        #x = [val*1e6 for val in x]
        # or x = list(map(lambda x: x*1e6, x))
        # List comprehension is more pythonic and it's usually faster
        # if we need to use lambdas in map

        # Values to plot on the graphic
        source = dict(x=x, y=y)

        # Add the fitnesses to the tooltips
        for j, fit in enumerate(fit_names):
            source.update({fit: [f"{eng_string(f[j])}{fit_units[j]}" for f in fits]})

        # Add the circuit variables to the tooltips
        # ind[j] é a variável 'j' do individuo
        for j, var in enumerate(vars_names):
            source.update({var: [f"{eng_string(ind[j])}{vars_units[j]}" for ind in inds]})

        p.circle('x', 'y', source=ColumnDataSource(data=source), size=10,
                 color=colors[idx], muted_color=colors[idx], muted_alpha=0.1,
                 legend=f"Pareto {idx+1} (ind={len(inds)})")

    # Format the title
    p.title.text_font_size = '16pt'
    p.title.align = 'center'
    # Format the axis labels
    p.xaxis.axis_label = f"{fit_names[0]} [{fit_units[0]}]"
    p.xaxis.formatter = PrintfTickFormatter(format='%.2e')
    p.yaxis.axis_label = f"{fit_names[1]} [{fit_units[1]}]"
    p.axis.axis_label_text_font_style = 'bold'
    p.axis.axis_label_text_font_size = '11pt'
    # Format the legend
    p.legend.location = "bottom_right"
    p.legend.click_policy = "mute"
    p.legend.label_text_font_size = '8pt'

    show(p)

## Plot Pareto Fronts (old version)

In [8]:
plot_pareto_fronts_old(fronts, circuit_vars, objectives, plot_fname='test old')

Loading BokehJS ...